In [1]:
#imports
import numpy as np
import pandas as pd
import sqlalchemy
import datetime

# Loading Data into DataFrames

In [2]:
appointments_data = pd.read_csv("Data_Files/jedi_appointments.csv")
establishments_data = pd.read_csv("Data_Files/jedi_establishments.csv")
speciality_data = pd.read_csv("Data_Files/jedi_speciality.csv")
traffic_data = pd.read_csv("Data_Files/jedi_traffic.csv")

In [3]:
# information for reference
print("Appointments table:\n", appointments_data.info())

print("Establishments table:\n", establishments_data.info())

print("Speciality table:\n", speciality_data.info())

print("Traffic table:\n", traffic_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76090 entries, 0 to 76089
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                76090 non-null  object
 1   created_at        76090 non-null  object
 2   appointment_from  76090 non-null  object
 3   status            76090 non-null  object
 4   type              76090 non-null  object
 5   source            76090 non-null  object
 6   establishment_id  76090 non-null  object
 7   patient_id        76090 non-null  object
 8   search_page       76090 non-null  object
 9   utm_flags         76090 non-null  object
dtypes: object(10)
memory usage: 5.8+ MB
Appointments table:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70752 entries, 0 to 70751
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        70752 non-null  object
 1   locality  70752 non-null  object
 2   city   

# Data Cleaning and Loading into SQL

In [4]:
# creating engine
engine = sqlalchemy.create_engine('mysql+pymysql://root:<pass>@localhost:3306/<dbname>')

In [5]:
# stripping establishments table and loading to SQL
df_obj = establishments_data.select_dtypes(['object'])
establishments_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
establishments_data['locality'] = establishments_data['locality'].str.encode('ascii', 'ignore').str.decode('utf-8')

In [6]:
# loading establishments data into sql
establishments_data.to_sql(
name = 'establishments',
con = engine,
index = False,
if_exists='replace'
)

In [7]:
# generating locality and city for appointments_data
establishments_data.rename(columns={'id': 'establishment_id'}, inplace=True)
appointments_data = pd.merge(left = appointments_data, right=establishments_data, left_on='establishment_id', right_on='establishment_id', how='left')
appointments_data.head()

,id,created_at,appointment_from,status,type,source,establishment_id,patient_id,search_page,utm_flags,locality,city
0,119cc56861f3e602654b325802c4e373d89c8d25,2020-12-12 15:02:49,2020-12-14 14:00:00,CONFIRMED,abs,android,b828d3b17d1c61cd3decb6d8ca9d62052f013cce,bed196062508864d9debbf7cd79bf0226416b65d,City|Ahmedabad|subspeciality|ophthalmologist/ ...,NULL|NULL,Navrangpura,Ahmedabad
1,4d2dc045f02d7b59988a1edcf9c6cef09cee18fe,2020-12-05 09:46:06,2020-12-05 11:30:00,CANCELLED,abs,mWeb,b828d3b17d1c61cd3decb6d8ca9d62052f013cce,f9973bdaf14a58e615bebd2caceece89513f4253,city|Ahmedabad|subspeciality|Ophthalmologist,NULL|NULL,Navrangpura,Ahmedabad
2,655188d31614f9757da766e4e9be61ebd2ec408b,2020-12-18 06:31:38,2020-12-18 13:45:00,CANCELLED,abs,mWeb,b828d3b17d1c61cd3decb6d8ca9d62052f013cce,41a5a7375a6b9d0231b7f1f6ea81372f61fd6b73,locality|Ambawadi|subspeciality|ophthalmologis...,NULL|NULL,Navrangpura,Ahmedabad
3,44c395026f72db339eb32cdc223935444033df44,2021-01-08 08:22:31,2021-01-08 11:30:00,CONFIRMED,abs,mWeb,b828d3b17d1c61cd3decb6d8ca9d62052f013cce,c5343acbd9cbb8852a66cb163eae519e9da5c5e1,city|Ahmedabad|subspeciality|Ophthalmologist,NULL|NULL,Navrangpura,Ahmedabad
4,cf481ac856902648b79a3a15060937b47bf551e4,2020-12-07 08:07:05,2020-12-09 04:00:00,CONFIRMED,abs,android,8777db3f7f72eb23e07c4d07006fc81fca9aee89,525ee5b7e54f68443ee7449b185f9b9d19351813,City|Thane|subspeciality|plastic surgeon,NULL|NULL,Thane West,Thane


In [8]:
# appointments typecasting
appointments_data['created_at'] = pd.to_datetime(appointments_data['created_at'])
appointments_data['appointment_from'] = pd.to_datetime(appointments_data['appointment_from'])

# setting index to normalized 'created_at'
appointments_data['ref_date'] = appointments_data['created_at'].dt.normalize()
# appointments_data.set_index('ref_date', inplace=True)

appointments_data['speciality'] = appointments_data['search_page'].str.split('|',expand=True)[3].apply(lambda x: x.lower())
appointments_data.drop(['search_page'], axis=1, inplace=True)

In [9]:
# appointments data for only confirmed bookings
confirmed_appointments_data = appointments_data[appointments_data['status'] == 'CONFIRMED']
confirmed_appointments_data['city'] = confirmed_appointments_data['city'].str.lower()

<ipython-input-9-a04fe99e61ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_appointments_data['city'] = confirmed_appointments_data['city'].str.lower()


In [10]:
# loading appointments to SQL
appointments_data.to_sql('appointments', engine, if_exists='replace')

In [11]:
# stripping speciality table
df_obj = speciality_data.select_dtypes(['object'])
speciality_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [12]:
# loading speciality table to SQL
speciality_data.to_sql('speciality', engine, if_exists='replace', index=False)

In [13]:
#traffic data - generating date stamp and dropping individual columns
traffic_data['date'] = pd.to_datetime(traffic_data[['year','month','day']])
traffic_data = traffic_data.drop(['year','month','day'], axis = 1)

In [14]:
# appointments data is available only for india, so reducing traffic_data parallely
traffic_data['country'] = traffic_data['country'].str.lower()

# getting all data from india, or from cities in establishment data if city is not specified
location_filter = (traffic_data['country'] == 'india') | ((traffic_data['country'].isnull().values.any()) & traffic_data.city.isin(establishments_data['city']))

traffic_data_india = traffic_data.loc[location_filter].drop(['country'], axis=1)

In [15]:
# stripping traffic table
df_obj = traffic_data.select_dtypes(['object'])
traffic_data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
traffic_data[df_obj.columns] = df_obj.apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('utf-8'))

In [16]:
# loading traffic table to SQL
traffic_data.to_sql('traffic', engine, if_exists='replace', index=False)

# Conversion

#### Overall

In [17]:
# on all of the data
total_traffic = traffic_data['dau'].sum()

total_booked = len(appointments_data.index)
total_confirmed = len(confirmed_appointments_data)

net_conversion = (total_confirmed/total_traffic)*100
gross_conversion = (total_booked/total_traffic)*100
print("net conversion = {:.2f} %".format(net_conversion))
print("gross conversion = {:.2f} %".format(gross_conversion))

net conversion = 1.93 %
gross conversion = 2.48 %


In [18]:
# using only India's traffic data to get a more accurate conversion measure
total_traffic_india = traffic_data_india['dau'].sum()

net_conversion_india = (total_confirmed/total_traffic_india)*100
gross_conversion_india = (total_booked/total_traffic_india)*100
print("net conversion = {:.2f} %".format(net_conversion_india))
print("gross conversion = {:.2f} %".format(gross_conversion_india))

net conversion = 2.00 %
gross conversion = 2.57 %


#### Daily Conversion

In [19]:
# conversion rate by daydaily_traffic = traffic_data_india.groupby(['date']).agg({'dau':'sum'})

daily_appointments = appointments_data['ref_date'].value_counts()
daily_confirmed_appointments = confirmed_appointments_data['ref_date'].value_counts()
daily_traffic = traffic_data_india.groupby(['date']).agg({'dau':'sum'})

daily_stats = pd.concat([daily_appointments, daily_confirmed_appointments, daily_traffic], axis='columns')
daily_stats.columns = ['total_booked', 'confirmed', 'traffic']

# daily_stats['gross conversion %'] = round((daily_stats['total_booked']/daily_stats['traffic'])*100,2)
# daily_stats['net conversion %'] = round((daily_stats['confirmed']/daily_stats['traffic'])*100,2)

In [20]:
# saving daily stats to csv
daily_stats.to_csv('Analysis/daily_conversion_stats.csv')

daily_stats.assign(gross_conversion = lambda x: round((x['total_booked']/x['traffic'])*100,2),
                  net_conversion = lambda x: round((x['confirmed']/x['traffic'])*100,2))

,total_booked,confirmed,traffic,gross_conversion,net_conversion
2020-12-01,1143,896,48341,2.36,1.85
2020-12-02,1098,855,47079,2.33,1.82
2020-12-03,1040,803,46461,2.24,1.73
2020-12-04,1200,930,46419,2.59,2.00
2020-12-05,1476,1151,49102,3.01,2.34
...,...,...,...,...,...
2021-01-27,1220,949,54449,2.24,1.74
2021-01-28,1089,845,45862,2.37,1.84
2021-01-29,1415,1117,43218,3.27,2.58
2021-01-30,1662,1308,44064,3.77,2.97


#### Weekly and Monthly

In [21]:
# weekly
daily_stats.resample('W').sum().assign(gross_conversion = lambda x: round((x['total_booked']/x['traffic'])*100,2),
                  net_conversion = lambda x: round((x['confirmed']/x['traffic'])*100,2))

,total_booked,confirmed,traffic,gross_conversion,net_conversion
2020-12-06,6917,5364,279274,2.48,1.92
2020-12-13,8364,6491,333663,2.51,1.95
2020-12-20,7559,6158,332037,2.28,1.85
2020-12-27,7812,6062,323053,2.42,1.88
2021-01-03,8450,6475,278418,3.04,2.33
2021-01-10,9751,7596,371151,2.63,2.05
2021-01-17,8895,6919,348893,2.55,1.98
2021-01-24,9144,7117,366836,2.49,1.94
2021-01-31,9198,7090,328824,2.80,2.16


In [22]:
# monthly
daily_stats.resample('M').sum().assign(gross_conversion = lambda x: round((x['total_booked']/x['traffic'])*100,2),
                  net_conversion = lambda x: round((x['confirmed']/x['traffic'])*100,2))

,total_booked,confirmed,traffic,gross_conversion,net_conversion
2020-12-31,35386,27750,1409068,2.51,1.97
2021-01-31,40704,31522,1553081,2.62,2.03


#### By Source

In [23]:
appointments_data['source'] = appointments_data['source'].str.lower()
total_appointments_by_source = appointments_data['source'].value_counts()
confirmed_by_source = confirmed_appointments_data['source'].value_counts()

traffic_data_india['platform'] = traffic_data_india['platform'].str.lower()
traffic_by_source = traffic_data_india.groupby(['platform']).agg({'dau':'sum'})

conversion_stats_source = pd.concat([total_appointments_by_source, confirmed_by_source, traffic_by_source], axis='columns').fillna(0)
conversion_stats_source.columns = ['total_appointments_by_source', 'confirmed_by_source', 'traffic_by_source']

conversion_stats_source.total_appointments_by_source = conversion_stats_source.total_appointments_by_source.astype(int)
conversion_stats_source.confirmed_by_source = conversion_stats_source.confirmed_by_source.astype(int)

conversion_stats_source.assign(gross_conversion = lambda x: round((x['total_appointments_by_source']/x['traffic_by_source'])*100,2),
                  net_conversion = lambda x: round((x['confirmed_by_source']/x['traffic_by_source'])*100,2))

,total_appointments_by_source,confirmed_by_source,traffic_by_source,gross_conversion,net_conversion
mweb,36112,0,1983162.0,1.82,0.00
android,28048,21284,464141.0,6.04,4.59
ios,11930,0,87930.0,13.57,0.00
mWeb,0,28588,0.0,NaN,inf
iOS,0,9400,0.0,NaN,inf
web,0,0,426916.0,0.00,0.00


In [24]:
# store in csv
conversion_stats_source.to_csv('Analysis/conversion_stats_by_source.csv')

#### By City

In [25]:
appointments_data['city'] = appointments_data['city'].str.lower()
total_appointments_by_city = appointments_data['city'].value_counts()
confirmed_by_city = confirmed_appointments_data['city'].value_counts()

traffic_data_india['city'] = traffic_data_india['city'].str.lower()
traffic_by_city = traffic_data_india.groupby(['city']).agg({'dau':'sum'})

conversion_stats_city = pd.concat([total_appointments_by_city, confirmed_by_city, traffic_by_city], axis='columns').fillna(0)
conversion_stats_city.columns = ['total_appointments_by_city', 'confirmed_by_city', 'traffic_by_city']

conversion_stats_city.total_appointments_by_city = conversion_stats_city.total_appointments_by_city.astype(int)
conversion_stats_city.confirmed_by_city = conversion_stats_city.confirmed_by_city.astype(int)

# dropping columns where no appointments have been done
conversion_stats_city = conversion_stats_city[conversion_stats_city['total_appointments_by_city'] > 0]

conversion_stats_city.assign(gross_conversion = lambda x: round((x['total_appointments_by_city']/x['traffic_by_city'])*100,2),
                  net_conversion = lambda x: round((x['confirmed_by_city']/x['traffic_by_city'])*100,2))

,total_appointments_by_city,confirmed_by_city,traffic_by_city,gross_conversion,net_conversion
bangalore,19954,15617,474855,4.20,3.29
delhi,9331,7040,370573,2.52,1.90
pune,8175,6641,261952,3.12,2.54
hyderabad,7991,6316,273769,2.92,2.31
gurgaon,6863,5310,94376,7.27,5.63
...,...,...,...,...,...
vizianagaram,1,1,206,0.49,0.49
tiruchirappalli,1,0,1042,0.10,0.00
karur,1,1,191,0.52,0.52
hisar,1,1,457,0.22,0.22


In [26]:
# store in csv
conversion_stats_city.to_csv('Analysis/conversion_stats_by_city.csv')

#### By Speciality

In [27]:
appointments_data['speciality'] = appointments_data['speciality'].str.lower()
total_appointments_by_speciality = appointments_data['speciality'].value_counts()
confirmed_by_speciality = confirmed_appointments_data['speciality'].value_counts()

traffic_data_india['query_value'] = traffic_data_india['query_value'].str.lower()
traffic_by_speciality = traffic_data_india.groupby(['query_value']).agg({'dau':'sum'})

conversion_stats_speciality = pd.concat([total_appointments_by_speciality, confirmed_by_speciality, traffic_by_speciality], axis='columns').fillna(0)
conversion_stats_speciality.columns = ['total_appointments_by_speciality', 'confirmed_by_speciality', 'traffic_by_speciality']

conversion_stats_speciality.total_appointments_by_speciality = conversion_stats_speciality.total_appointments_by_speciality.astype(int)
conversion_stats_speciality.confirmed_by_speciality = conversion_stats_speciality.confirmed_by_speciality.astype(int)
conversion_stats_speciality.traffic_by_speciality = conversion_stats_speciality.traffic_by_speciality.astype(int)

# dropping columns where no appointments have been done
conversion_stats_speciality = conversion_stats_speciality[conversion_stats_speciality['total_appointments_by_speciality'] > 0]

conversion_stats_speciality.assign(gross_conversion = lambda x: round((x['total_appointments_by_speciality']/x['traffic_by_speciality'])*100,2),
                  net_conversion = lambda x: round((x['confirmed_by_speciality']/x['traffic_by_speciality'])*100,2))

,total_appointments_by_speciality,confirmed_by_speciality,traffic_by_speciality,gross_conversion,net_conversion
dermatologist,11041,8893,328626,3.36,2.71
gynecologist/obstetrician,8880,6911,265586,3.34,2.60
dentist,8567,6881,214026,4.00,3.22
orthopedist,6483,5218,185704,3.49,2.81
general physician,4916,3748,192784,2.55,1.94
...,...,...,...,...,...
pediatric endocrinologist,1,1,73,1.37,1.37
infectious diseases physician,1,1,0,inf,inf
hemorrhoids,1,1,7,14.29,14.29
penis pain,1,0,15,6.67,0.00


In [28]:
# store in csv
conversion_stats_speciality.to_csv('Analysis/conversion_stats_by_speciality.csv')

# Retention

### Overall Retention for given data

In [29]:
def total_retention(df):
    unique_users = len(df['patient_id'].unique())

    filter_duplicate = df['patient_id'].duplicated(keep=False)

    unique_retained_users = len(df[filter_duplicate]['patient_id'].unique())
    churned_users = unique_users - unique_retained_users
    
    net_retention = (unique_retained_users / unique_users) * 100
    return net_retention

In [30]:
# gross and net total retention
gross_total_retention = total_retention(appointments_data)
net_total_retention = total_retention(confirmed_appointments_data)

print("gross total retention = {:.2f} %".format(gross_total_retention))
print("net total retention = {:.2f} %".format(net_total_retention))

gross total retention = 3.19 %
net total retention = 2.20 %


In [31]:
filter_duplicate = appointments_data['patient_id'].duplicated(keep=False)

# most frequent users
most_frequent_users = appointments_data[filter_duplicate]['patient_id'].value_counts().nlargest(10)

### Retention over defined offset period

In [32]:
def retained(df, user_id, offset):
    user_data = df[df['patient_id'] == user_id].sort_values(by=['ref_date'],ascending=True)
    td = user_data.iloc[1]['ref_date'] - user_data.iloc[0]['ref_date']
    return td.days <= offset

def calculate_retention(df, offset = 30):
    repeat_users = df[df['patient_id'].duplicated(keep=False)]['patient_id'].unique()
    unique_users = len(df['patient_id'].unique())
    
    retained_count = 0
    for user in repeat_users:
        if retained(df, user, offset):
            retained_count+=1
    retention_rate = (retained_count/unique_users) * 100
    return round(retention_rate, 2)

In [33]:
# gross 30-day retention
gross_retention_30 = calculate_retention(appointments_data, 30)

# net 30-day retention
net_retention_30 = calculate_retention(confirmed_appointments_data, 30)

print("gross 30-day retention = {:.2f} %".format(gross_retention_30))
print("net 30-day retention = {:.2f} %".format(net_retention_30))

gross 30-day retention = 2.99 %
net 30-day retention = 2.02 %


#### Validating the function by calculating 60-day retention, which should be the same as total retention, given that is our entire data.
###### As we can see below, the retention rate is the same.

In [34]:
# gross 60-day retention
gross_retention_60 = calculate_retention(appointments_data, 60)

# net 60-day retention
net_retention_60 = calculate_retention(confirmed_appointments_data, 60)

print("gross 60-day retention = {:.2f} %".format(gross_retention_60))
print("net 60-day retention = {:.2f} %".format(net_retention_60))

gross 60-day retention = 3.19 %
net 60-day retention = 2.20 %


In [35]:
len(appointments_data[appointments_data['patient_id'].duplicated(keep=False)]['patient_id'].unique())

2332

### Function to calculate over cuts

In [36]:
def calculate_retention_by_parameter(df, parameter, offset = 30):
    groups = df.groupby([parameter])

    retention_by_parameter = pd.DataFrame(columns = [parameter, 'retention_rate'])

    # calculating retention for parameter only if number of bookings > 10
    for name, group in groups:
        if len(group.index) > 10:
            r_rate = calculate_retention(group, offset)
            retention_by_parameter = retention_by_parameter.append({parameter: name, 'retention_rate': r_rate},
                                                                  ignore_index = True)
    return retention_by_parameter

#### Calculating over cities

In [37]:
# calculating 30-day retention rates by cities if number of appointments in city > 10

# gross
gross_retention_by_city_30 = calculate_retention_by_parameter(appointments_data, 'city', 30)

# net
net_retention_by_city_30 = calculate_retention_by_parameter(confirmed_appointments_data, 'city', 30)

# combined
retention_by_city = pd.merge(gross_retention_by_city_30, net_retention_by_city_30, on='city')
retention_by_city.columns = ['city', 'gross_retention_by_city', 'net_retention_by_city']

In [38]:
# storing in csv
retention_by_city.to_csv('Analysis/retention_by_city.csv', index=False)

#### Calculating over specialities

In [39]:
# calculating 30-day retention rates by speciality if number of appointments for speciality > 10

# gross
gross_retention_by_speciality_30 = calculate_retention_by_parameter(appointments_data, 'speciality', 30)

# net
net_retention_by_speciality_30 = calculate_retention_by_parameter(confirmed_appointments_data, 'speciality', 30)

# combined
retention_by_speciality = pd.merge(gross_retention_by_speciality_30, net_retention_by_speciality_30, on='speciality')
retention_by_speciality.columns = ['speciality', 'gross_retention_by_speciality', 'net_retention_by_speciality']

In [40]:
retention_by_speciality.to_csv('Analysis/retention_by_speciality.csv', index=False)

### Calculating over source

In [41]:
# calculating 30-day retention rates by source if number of appointments for speciality > 10

# gross
gross_retention_by_source_30 = calculate_retention_by_parameter(appointments_data, 'source', 30)

# net
net_retention_by_source_30 = calculate_retention_by_parameter(confirmed_appointments_data, 'source', 30)

# combined
retention_by_source = pd.merge(gross_retention_by_source_30, net_retention_by_source_30, on='source')
retention_by_source.columns = ['source', 'gross_retention_by_source', 'net_retention_by_source']

In [42]:
retention_by_source.to_csv('Analysis/retention_by_source.csv', index=False)